### Section 0: Loading packages and testing earth engine connection

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ee
import earthlib as eli
import geemap

In [ ]:
# OPTIONAL: authenticate earth engine api access (not required on every run)
# ee.Authenticate()

# initialize earth engine api access (required on every run)
ee.Initialize()

# test whether the initialization was successful by printing the elevation of Mount Everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Point location elevation (m):', elev)

### Section 1: Dataset preparation

In [ ]:
# select the sensor to use
sensor = 'Landsat8'
collection = eli.getCollection(sensor)
bands = eli.getBands(sensor)
scaler = eli.Scale.bySensor(sensor)
cloudMask = eli.CloudMask.bySensor(sensor)

# set the dates
startDate = "2021-01-01";
endDate = "2021-09-01";

# get a land mask to use
land = ee.Image("UMD/hansen/global_forest_change_2019_v1_7").select(["datamask"]).eq(1)

# create and filter the image collection
filtered = collection.filterDate(startDate, endDate).map(cloudMask).select(bands)

# create a median composite
sr = filtered.map(scaler).median()

### Section 2: Mapping the input data

In [ ]:
# set SR visualization parameters.
visBands = ["SR_B6", "SR_B5", "SR_B4"] # Landsat-8
#visBands = ["Nadir_Reflectance_Band6", "Nadir_Reflectance_Band2", "Nadir_Reflectance_Band1"] # MODIS
#visBands = ["B11", "B8", "B4"] # Sentinel-2
visParams = {
  'min': 0,
  'max': 0.5,
  'bands': visBands,
}

# Create a folium map object.
Map = geemap.Map(center=[37.5, -118], zoom=6)

# Add the image to the map object.
Map.addLayer(sr, visParams, sensor)

# Display the map.
display(Map)

### Section 3: Estimating pixel-level fractional cover

In [ ]:
SoilPVNPV = eli.SoilPVNPV.bySensor(sensor)
unmixed = SoilPVNPV(sr)

visParams = {'min': [0, 0.1, 0], 'max': [0.8, 1, 0.8]}
Map.addLayer(unmixed.select([0,1,2]), visParams, 'unmixed')

# the map object from the cell above should update